# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
vacation = "output/matchcity.csv"
vacation_df = pd.read_csv(vacation)
vacation_df

,City,Temperature,Latitude,City_ID,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,dikson,14.29,73.51,66.0,12.0,RU,1.558379e+09,85.0,73.51,80.55,20.22,12.15
1,hermanus,67.39,-34.42,40.0,9.0,ZA,1.558379e+09,74.0,-34.42,19.24,55.00,23.78
2,mataura,57.99,-46.19,96.0,58.0,NZ,1.558379e+09,88.0,-46.19,168.86,48.00,1.99
3,bambous virieux,77.49,-20.34,18.0,75.0,MU,1.558379e+09,74.0,-20.34,57.76,78.80,11.41
4,rikitea,77.04,-23.12,28.0,95.0,PF,1.558379e+09,76.0,-23.12,-134.97,74.58,12.82
...,...,...,...,...,...,...,...,...,...,...,...,...
199,yulara,57.20,-25.24,303.0,20.0,AU,1.558379e+09,77.0,-25.24,130.99,62.60,8.16
200,dunedin,55.00,-45.87,307.0,1.0,NZ,1.558379e+09,81.0,-45.87,170.50,46.99,4.00
201,mecca,67.93,21.43,525.0,1.0,SA,1.558379e+09,48.0,21.43,39.83,88.62,1.74
202,salinopolis,82.08,-0.61,305.0,90.0,BR,1.558379e+09,82.0,-0.61,-47.36,82.14,12.57


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [14]:
# Store latitude and longitude in locations
locations = vacation_df[["Lat", "Lng"]]

In [15]:
humidity = vacation_df["Humidity"].astype(float)

In [39]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
warm = vacation_df.loc[vacation_df['Max Temp'] > 70]
warmcool = warm.loc[warm['Max Temp'] < 80]
warmcoolstill = warmcool.loc[warmcool['Wind Speed'] < 10]
warmcoolstillclear = warmcoolstill.loc[warmcoolstill['Cloudiness'] == 0]
warmcoolstillclear

,City,Temperature,Latitude,City_ID,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
22,vaini,78.80,-21.20,139.0,0.0,IN,1.558379e+09,88.0,15.34,74.49,78.90,1.59
123,ponta do sol,58.55,32.67,70.0,0.0,BR,1.558379e+09,57.0,-20.63,-46.00,73.68,5.32
157,cidreira,67.53,-30.18,88.0,0.0,BR,1.558379e+09,66.0,-30.17,-50.22,76.56,5.50


In [28]:
# Export file as a CSV, without the Pandas index, but with the header
warmcoolstillclear.to_csv("Output/perfectcity.csv", index=False, header=True)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
hotel_df = warmcoolstillclear

In [38]:
hotel = ["Hotel 1","Hotel 2","Hotel 3"]
hotel_df["Hotel Name"] = hotel
hotel_df

C:\Users\drewl\Anaconda3\envs\PythonData2020_1\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Temperature,Latitude,City_ID,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
22,vaini,78.80,-21.20,139.0,0.0,IN,1.558379e+09,88.0,15.34,74.49,78.90,1.59,Hotel 1
123,ponta do sol,58.55,32.67,70.0,0.0,BR,1.558379e+09,57.0,-20.63,-46.00,73.68,5.32,Hotel 2
157,cidreira,67.53,-30.18,88.0,0.0,BR,1.558379e+09,66.0,-30.17,-50.22,76.56,5.50,Hotel 3


In [41]:
# geocoordinates
target_coordinates = "15.34, 74.49"
target_search = "Hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)


In [43]:
# convert response to json
places_data = response.json()

In [44]:
print(places_data["results"][0]["name"])
print(places_data["results"][0]["vicinity"])

Dandeli Mist Jungle Stay
12km from Ramnagar, Chandwadi road, Shinoli, Joida, Near Jagelbet Republic of India, SH 34, Bori


In [45]:
# geocoordinates
target_coordinates = "-20.63,-46.00"
target_search = "Hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [46]:
# convert response to json
places_data = response.json()

In [47]:
print(places_data["results"][0]["name"])
print(places_data["results"][0]["vicinity"])

Hotel Escarpas do Lago
Av. das Caravelas, 25 - Engenheiro José Mendes Júnior, Capitólio


In [48]:
# geocoordinates
target_coordinates = "-30.17,-50.22"
target_search = "Hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [49]:
# convert response to json
places_data = response.json()

In [50]:
print(places_data["results"][0]["name"])
print(places_data["results"][0]["vicinity"])

Pond Country Club
Estrada RS 784 KM 7 - Lagoa, Cidreira


In [51]:
hotel = ["Dandeli Mist Jungle Stay","Hotel Escarpas do Lago","Pond Country Club"]
hotel_df["Hotel Name"] = hotel
hotel_df

C:\Users\drewl\Anaconda3\envs\PythonData2020_1\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Temperature,Latitude,City_ID,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
22,vaini,78.80,-21.20,139.0,0.0,IN,1.558379e+09,88.0,15.34,74.49,78.90,1.59,Dandeli Mist Jungle Stay
123,ponta do sol,58.55,32.67,70.0,0.0,BR,1.558379e+09,57.0,-20.63,-46.00,73.68,5.32,Hotel Escarpas do Lago
157,cidreira,67.53,-30.18,88.0,0.0,BR,1.558379e+09,66.0,-30.17,-50.22,76.56,5.50,Pond Country Club


In [52]:
# Export file as a CSV, without the Pandas index, but with the header
hotel_df.to_csv("Output/hotelcity.csv", index=False, header=True)

In [54]:
narrowed_city_df = warmcoolstillclear

In [55]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [56]:
# Create a list containing coordinates
coordinates = [
    (15.34, 74.49),
    (-20.63,-46.00),
    (-30.17,-50.22)
]
coordinates

[(15.34, 74.49), (-20.63, -46.0), (-30.17, -50.22)]

In [57]:
# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [58]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(coordinates)

# Display Map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…